In [ ]:
# Setup
import librosa
import numpy as np
import os
from pymusickit.key_finder import KeyFinder
import ipywidgets as widgets
import math

In [ ]:
from vocal_remover import direct_call_on_audio_array
from decompose import run_decomposer
from download_audio import save_youtube_audio, write_audio
from notebook_helpers import display_audio
from combine import *

# Mixing Tool

This notebook demonstrates the main results of the project. It is possible to mix together different songs, extract layers and modify them.

To set up the project follow instructions given in the README.md file.

How to use this notebook: read the explanations and run cells one by one.

## 1. Choose input audio files

Provide links to two audio files ...

In [ ]:
# Add YouTube links for 2 songs you want to mix
url1 = '...'
url2 = '...'

# Saving and loading audio files
audiopath1 = save_youtube_audio(url1, 'audio1')
audiopath2 = save_youtube_audio(url2, 'audio2')

song1 = KeyFinder(audiopath1)
song2 = KeyFinder(audiopath2)

audio1, sr1 = song1.waveform, song1.sr
audio2, sr2 = song2.waveform, song2.sr

Now, specify the timestamps of target audio and source audio ...

In [ ]:
a1_len_sec = librosa.get_duration(y=audio1, sr=sr1) # in seconds
a2_len_sec = librosa.get_duration(y=audio2, sr=sr2)

ts1 = widgets.FloatRangeSlider(
    value=[0, a1_len_sec],
    min=0, max=a1_len_sec, step=1.0,
    description='Audio 1:',
    readout=True,
    readout_format='.1f',
)
display(ts1)

ts2 = widgets.FloatRangeSlider(
    value=[0, a2_len_sec],
    min=0, max=a2_len_sec, step=1.0,
    description='Audio 2:',
    readout=True,
    readout_format='.1f',
)
display(ts2)

In [ ]:
a1_ts = [ts1.value, ts1.value[1]-ts1.value[0]]
a2_ts = [ts2.value, ts2.value[1]-ts2.value[0]]

audio1 = audio1[int(a1_ts[0][0]*sr1):int(a1_ts[0][1]*sr1)]
audio2 = audio2[int(a2_ts[0][0]*sr2):int(a2_ts[0][1]*sr2)]

In [ ]:
# Extracting layers
fore1, back1 = direct_call_on_audio_array(audio1, sr1)
fore2, back2 = direct_call_on_audio_array(audio2, sr2)

In [ ]:
back_path1 = os.path.join('audio', 'back1.wav')
back_path2 = os.path.join('audio', 'back2.wav')

write_audio(back_path1, back1.T, sr1)
write_audio(back_path2, back2.T, sr2)
_11,_12,perc1 = run_decomposer(back_path1, sr1)
_21,_22,perc2 = run_decomposer(back_path2, sr2)

In [ ]:
shifted_audio = shift_to_match_target_key(song2, song1, fore2, sr2)

## 2. Listen to extracted stems and mix

Listen to the extracted background, vocal parts and rhythm...

In [ ]:
print('Backing audio:')
display_audio(back1, sr1)

In [ ]:
print('Shifted vocals to match:')
display_audio(shifted_audio, sr1)

In [ ]:
print('Percussion track 1:')
display_audio(perc1, sr1)

Now choose which stems do you want to mix together and configure the volume of each one...

In [ ]:
print('VOLUME CONFIGURATION')

print('Backing:')
s1 = widgets.FloatSlider(min=0.0, max=1.0, step=0.1, value=1.0)
display(s1)

print('Shifted vocals:')
s2 = widgets.FloatSlider(min=0.0, max=1.0, step=0.1, value=1.0)
display(s2)

In [ ]:
mixed = combine_audio_layers([back1, shifted_audio], [s1.value, s2.value])
print('Combined audio:')
display_audio(mixed, sr1)

If you want to adjust the volume further, make sure to run the previous two cells in the same order again until you're satisfied with the mix ...

If you want to match the tempo run following cells...

In [ ]:
matched_tempo = match_tempo(shifted_audio, sr2, back1, sr1)

In [ ]:
final = combine_audio_layers([back1, matched_tempo], [s1.value, s2.value])
display_audio(final, sr1)